In [1]:
import os
import json
import zipfile
import pandas as pd
import json

In [26]:
def formatted_date(value):
    date_formatted = value.replace(".", "-")
    return date_formatted

In [ ]:
##getting charts

apple_charts = pd.read_csv("/Users/prayagbhatia/Desktop/ASU_Courses/SWM/project/CHARTS/APPLE1440.csv",header=None)
chart_column_names = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
apple_charts.columns = chart_column_names
apple_charts['DateFormatted'] = apple_charts['Date'].apply(formatted_date).apply(pd.to_datetime).dt.date
apple_charts

In [ ]:
amazon_charts = pd.read_csv("/Users/prayagbhatia/Desktop/ASU_Courses/SWM/project/CHARTS/AMAZON1440.csv",header=None)
chart_column_names = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
amazon_charts.columns = chart_column_names
amazon_charts['DateFormatted'] = amazon_charts['Date'].apply(formatted_date).apply(pd.to_datetime).dt.date
amazon_charts

amazon_charts.iloc[1084]

In [ ]:
### Mapping Amazon news to amazon charts taking into consideration the Open values
labels = []
for index , row in amazon_news_df.iterrows():
    news_date = row.Date
    match_date_condition = amazon_charts['DateFormatted'] == news_date
    match_idx = amazon_charts.loc[match_date_condition].index
    if len(match_idx):
        match_idx = match_idx[0]
        if amazon_charts.iloc[match_idx-1].Open < amazon_charts.iloc[match_idx].Open:
            labels.append(1)
        else:
            labels.append(0)
    else:
        labels.append(None)
        

amazon_news_df['labels'] = labels
amazon_news_df


In [ ]:
### Mapping Apple news to apple charts taking into consideration the Open values
labels = []
for index , row in apple_news_df.iterrows():
    news_date = row.Date
    match_date_condition = apple_charts['DateFormatted'] == news_date
    match_idx = apple_charts.loc[match_date_condition].index
    if len(match_idx):
        match_idx = match_idx[0]
        if apple_charts.iloc[match_idx-1].Open < apple_charts.iloc[match_idx].Open:
            labels.append(1)
        else:
            labels.append(0)
    else:
        labels.append(None)
        

apple_news_df['labels'] = labels
apple_news_df

In [38]:
scope = ['https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/prayagbhatia/Downloads/creds.json', scope)
gauth = GoogleAuth()
gauth.credentials = credentials

# Create the PyDrive client
drive = GoogleDrive(gauth)



In [ ]:
csv_string = apple_news_df.to_csv()


file = drive.CreateFile({'title': 'applenews3.csv'})
file.SetContentString(csv_string)
file.Upload()


In [22]:
apple_news_df2 = apple_news_df.drop('text', axis=1)
apple_news_df2.to_csv('apple_news2.csv')

In [23]:
amazon_news_df2 = amazon_news_df.drop('text', axis=1)
amazon_news_df2.to_csv('amazon_news2.csv')